In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from scipy.optimize import minimize
from sklearn.metrics import log_loss
import matplotlib.pylab as plt

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import optimizers

plt.style.use('ggplot')

In [ ]:
train = pd.read_csv('train.csv', delimiter=';')
test  = pd.read_csv('test.csv', delimiter=';')

test.replace({'None' : 999}, inplace=True)
test[['smoke', 'alco', 'active']] = test[['smoke', 'alco', 'active']].astype(int)

test.replace({999 : np.nan}, inplace=True)
test[['gluc', 'cholesterol']] = test[['gluc', 'cholesterol']].astype(int)

N_train = train.shape[0]

data = pd.concat([train, test], 0).reset_index(drop=True)

data['imt'] = data['weight'] / (data['height'] / 100) ** 2

data['chol+gluc'] = data['cholesterol'].astype(str) + data['gluc'].astype(str) 
data['chol+gluc'] = pd.factorize(data['chol+gluc'])[0]

In [ ]:
clean = False
if clean:
    bad_index = (data.height < 110) & (data.weight > 150)
    buf = data.loc[bad_index, 'height']
    data.loc[bad_index, 'height'] = data.loc[bad_index, 'weight']
    data.loc[bad_index, 'weight'] = buf
    
    data.loc[data.height < 90, 'height'] += 100
    
    data.loc[(data.height >= 100) & (data.height < 110), 'height'] += 60 
    
    data.loc[(data.height >= 110) & (data.height < 120) & (data.weight < 110), 'height'] += 60

    data.loc[(data.height == data.ap_hi) & (data.weight == data.ap_lo), 'height'] = np.nan
    data.loc[(data.height == data.ap_hi) & (data.weight == data.ap_lo), 'weight'] = np.nan
    
    data.loc[data.height <= 130, ['weight', 'height']] = [np.nan, np.nan]
    data.loc[(data.height <= 135) & (data.weight > 60), 'height'] = np.nan
    data.loc[(data.height < 140) & (data.weight > 80), 'height'] = np.nan
    data.loc[data.height > 210, 'height'] = np.nan
    
    data.loc[data.weight < 20, 'weight'] *= 10
    
    data.loc[data.weight <= 30, 'weight'] = np.nan
    data.loc[(data.weight < 36) & (data.height > 160), 'weight'] = np.nan
    
    data.loc[(data.weight > 140) & (data.height == data.weight), 'weight'] = np.nan
    
    data.loc[data.weight > 125, 'weight'] = np.nan
    
    data.replace({'weight': {120.0: np.nan}}, inplace=True)
    
    data['ap_hi'] = np.abs(data['ap_hi'])
    data['ap_lo'] = np.abs(data['ap_lo'])
    
    data.loc[data.ap_lo > 5000, 'ap_lo'] /= 100
    data.loc[data.ap_hi > 5000, 'ap_hi'] /= 100
    data.loc[data.ap_hi > 250, 'ap_hi'] /= 10
    data.loc[data.ap_lo > 250, 'ap_lo'] /= 10
    
    data.loc[[12494, 60477, 51749], ['ap_hi', 'ap_lo']] = [np.nan, np.nan]
    data.loc[[75399], 'ap_lo'] = 200
    data.loc[6580, ['ap_hi', 'ap_lo']] = [np.nan, np.nan]
    data.loc[data.ap_hi < 10, 'ap_hi'] = np.nan
    
    data.loc[data.ap_hi == 906, ['ap_hi', 'ap_lo']] = [np.nan, np.nan]
    data.loc[data.ap_hi == 90.6, ['ap_hi', 'ap_lo']] = [np.nan, np.nan]
    data.loc[data.ap_hi == 701, ['ap_hi', 'ap_lo']] = [110, 70]
    data.loc[data.ap_hi == 309, ['ap_hi', 'ap_lo']] = [np.nan, np.nan]
    data.loc[data.ap_hi == 30.9, ['ap_hi', 'ap_lo']] = [np.nan, np.nan]
    data.loc[data.ap_hi == 806, ['ap_hi', 'ap_lo']] = [np.nan, np.nan]
    data.loc[data.ap_hi == 509, ['ap_hi', 'ap_lo']] = [np.nan, np.nan]
    data.loc[data.ap_hi == 50.9, ['ap_hi', 'ap_lo']] = [np.nan, np.nan]
    data.loc[data.ap_hi == 50.9, ['ap_hi', 'ap_lo']] = [np.nan, np.nan]
    data.loc[data.ap_hi == 40.1, ['ap_hi', 'ap_lo']] = [np.nan, np.nan]
    
    data.loc[data.ap_hi == 400, 'ap_hi'] = 100
    data.loc[data.ap_hi == 401, 'ap_hi'] = 101
    data.loc[data.ap_hi == 410, 'ap_hi'] = 110
    data.loc[data.ap_hi == 470, 'ap_hi'] = 170
    data.loc[data.ap_lo == 410, 'ap_lo'] = 110
    data.loc[data.ap_lo == 470, 'ap_lo'] = 170
    
    data.loc[(data.ap_hi == 138) & (data.ap_lo == 0), ['ap_hi', 'ap_lo']] = [np.nan, np.nan]
    data.loc[(data.ap_hi == 149) & (data.ap_lo == 0), ['ap_hi', 'ap_lo']] = [np.nan, np.nan]
    data.loc[(data.ap_hi == 90.7) & (data.ap_lo == 0), ['ap_hi', 'ap_lo']] = [np.nan, np.nan]
    data.loc[(data.ap_hi == 148) & (data.ap_lo == 0), ['ap_hi', 'ap_lo']] = [np.nan, np.nan]
    data.loc[(data.ap_hi == 80.6) & (data.ap_lo == 0), ['ap_hi', 'ap_lo']] = [np.nan, np.nan]
    data.loc[(data.ap_hi == 108) & (data.ap_lo == 0), ['ap_hi', 'ap_lo']] = [np.nan, np.nan]
    data.loc[(data.ap_hi == 121) & (data.ap_lo == 0), ['ap_hi', 'ap_lo']] = [np.nan, np.nan]
    data.loc[(data.ap_hi == 117) & (data.ap_lo == 0), ['ap_hi', 'ap_lo']] = [np.nan, np.nan]
    data.loc[(data.ap_hi == 118) & (data.ap_lo == 0), ['ap_hi', 'ap_lo']] = [np.nan, np.nan]
    
    data.loc[data.ap_lo == 0, 'ap_lo'] = np.nan
    
    data.replace({'ap_lo': {1: 100}}, inplace=True)
    
    data.loc[data.ap_hi < 30, 'ap_hi'] *= 10
    data.loc[data.ap_lo < 20, 'ap_lo'] *= 10

    data.replace({'ap_lo': {20: np.nan, 30: 80}}, inplace=True)
    
    bad_index = data.ap_hi < data.ap_lo
    buf = data.loc[bad_index, 'ap_hi']
    data.loc[bad_index, 'ap_hi'] = data.loc[bad_index, 'ap_lo']
    data.loc[bad_index, 'ap_lo'] = buf

In [ ]:
mean_y = data.groupby(['cholesterol'])['cardio'].mean().reset_index()
mean_y = mean_y.rename(columns = {'cardio' : 'p_chol'})
data = pd.merge(data, mean_y, on = 'cholesterol', how = 'left')

mean_y = data.groupby(['gluc'])['cardio'].mean().reset_index()
mean_y = mean_y.rename(columns = {'cardio' : 'p_gluc'})
data = pd.merge(data, mean_y, on = 'gluc', how = 'left')

mean_y = data.groupby(['chol+gluc'])['cardio'].mean().reset_index()
mean_y = mean_y.rename(columns = {'cardio' : 'p_ch+gl'})
data = pd.merge(data, mean_y, on = 'chol+gluc', how = 'left')

In [ ]:
cat_feat = ['alco', 'smoke', 'active', 'gluc', 'cholesterol', 'gender', 'chol+gluc']
for col in cat_feat:
    dummy = pd.get_dummies(data[col], prefix = col)
    data = pd.concat([data, dummy], 1)
data.drop(cat_feat, 1, inplace=True)

In [ ]:
feat = ['age', 'ap_hi', 'ap_lo', 'height','weight', 'imt']
for col in feat:
    data[feat] = (data[feat] - data[feat].mean())/data[feat].std()

In [ ]:
data = data.fillna(data.mean())

In [ ]:
data.head()

In [ ]:
##### for submit
x_train = data[:N_train].drop(['id', 'cardio'], 1).values
y_train = data[:N_train]['cardio'].values

x_test = data[N_train:].drop(['id', 'cardio'], 1).values
##### for cv
#x_train = data.loc[:int(0.8*N_train)].drop(['id', 'cardio'], 1).values
#y_train = data.loc[:int(0.8*N_train)]['cardio'].values

#x_test = data.loc[int(0.8*N_train):N_train-1].drop(['id', 'cardio'], 1).values
#y_test = data.loc[int(0.8*N_train):N_train-1]['cardio'].values

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.05)
model.add(Dense(32, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size=1024, epochs = 10, verbose=0)
y1 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y1)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.05)
model.add(Dense(32, activation='sigmoid', input_dim=x_train.shape[1]))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size = 1024, epochs = 10, verbose=0)
y2 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y2)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.01)
model.add(Dense(64, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size=1024, epochs = 20, verbose=0)
y3 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y3)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.01)
model.add(Dense(64, activation='sigmoid', input_dim=x_train.shape[1]))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(16, activation='sigmoid'))
model.add(Dense(8, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size=1024, epochs = 20, verbose=0)
y4 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y4)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.01)
model.add(Dense(12, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(32, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size=1024, epochs = 20, verbose=0)
y5 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y5)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.03)
model.add(Dense(12, activation='sigmoid', input_dim=x_train.shape[1]))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(12, activation='sigmoid'))
model.add(Dense(6, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size=1024, epochs = 20, verbose=0)
y6 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y6)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.03)
model.add(Dense(128, activation='sigmoid', input_dim=x_train.shape[1]))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size=1024, epochs = 20, verbose=0)
y7 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y7)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.03)
model.add(Dense(128, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size=1024, epochs = 20, verbose=0)
y8 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y8)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.03)
model.add(Dense(128, activation='sigmoid', input_dim=x_train.shape[1]))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(4, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size=1024, epochs = 20, verbose=0)
y9 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y9)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.03)
model.add(Dense(64, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size=1024, epochs = 20, verbose=0)
y10 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y10)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.03)
model.add(Dense(64, activation='sigmoid', input_dim=x_train.shape[1]))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size=1024, epochs = 20, verbose=0)
y11 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y11)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.01)
model.add(Dense(20, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(40, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(60, activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size = 1034, epochs = 20, verbose=0)
y12 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y12)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.01)
model.add(Dense(10, activation='sigmoid', input_dim=x_train.shape[1]))
model.add(Dense(30, activation='sigmoid'))
model.add(Dense(50, activation='sigmoid'))
model.add(Dropout(0.1))
model.add(Dense(50, activation='sigmoid'))
model.add(Dense(30, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size = 1024, epochs = 10, verbose=0)
y13 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y13)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.01)
model.add(Dense(10, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(30, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(50, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size = 1024, epochs = 20, verbose=0)
y14 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y14)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.01)
model.add(Dense(10, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(30, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size = 1024, epochs = 10, verbose=0)
y15 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y15)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.01)
model.add(Dense(10, activation='sigmoid', input_dim=x_train.shape[1]))
model.add(Dense(30, activation='sigmoid'))
model.add(Dense(30, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size = 1024, epochs = 10, verbose=0)
y16 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y16)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.01)
model.add(Dense(10, activation='tanh', input_dim=x_train.shape[1]))
model.add(Dense(30, activation='tanh'))
model.add(Dense(30, activation='tanh'))
model.add(Dense(10, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size = 1024, epochs = 10, verbose=0)
y17 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y17)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.01)
model.add(Dense(50, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(50, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size = 1024, epochs = 10, verbose=0)
y18 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y18)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.01)
model.add(Dense(50, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size = 1024, epochs = 10, verbose=0)
y19 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y19)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.01)
model.add(Dense(20, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(20, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size = 1024, epochs = 10, verbose=0)
y20 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y20)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.01)
model.add(Dense(50, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size = 1024, epochs = 10, verbose=0)
y21 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y21)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.01)
model.add(Dense(90, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size = 1024, epochs = 10, verbose=0)
y22 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y22)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.01)
model.add(Dense(20, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(60, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size = 1024, epochs = 10, verbose = 0)
y23 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y23)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.01)
model.add(Dense(60, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(60, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size = 1024, epochs = 10, verbose = 0)
y24 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y24)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.01)
model.add(Dense(20, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size = 1024, epochs = 10, verbose = 0)
y25 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y25)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.01)
model.add(Dense(20, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(20, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size = 1024, epochs = 15, verbose = 0)
y26 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y26)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.01)
model.add(Dense(20, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(60, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size = 1024, epochs = 10, verbose = 0)
y27 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y27)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.01)
model.add(Dense(60, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(40, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size = 1024, epochs = 15, verbose = 0)
y28 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y28)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.01)
model.add(Dense(20, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(40, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size = 1024, epochs = 10, verbose = 0)
y29 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y29)
del model

In [ ]:
model = Sequential()
opt = optimizers.adam(lr = 0.01)
model.add(Dense(40, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(40, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy'])

model.fit(x_train, y_train, batch_size = 1024, epochs = 10, verbose = 0)
y30 = model.predict_proba(x_test)
print 
#print log_loss(y_test, y30)
del model

In [ ]:
# cv
y_nn = ((y1 + y2 + y3 + y4 + y5 + y6 + y7 + y8 + y9 + y10\
        + y11 + y12 + y13 + y14 + y15 + y16 + y17 + y18 + y19 + y20\
        + y21 + y22 + y23 + y24 + y25 + y26 + y27 + y28 + y29 + y30)/30).T[0]
#log_loss(y_test, y_nn)

In [ ]:
y_nn

In [ ]:
output = pd.DataFrame(y_nn)
output.to_csv('nn_30.csv', header=None, index=None)
output.head(7)